In [1]:
import joblib

MODEL_PATH = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Chargement
artifacts = joblib.load(MODEL_PATH)

# Inspection des clés principales
print("🔍 Clés disponibles dans artifacts :", artifacts.keys())

# Si metadata est bien là, on regarde ce qu'il contient
metadata = artifacts.get("metadata", {})
print("🔍 Clés disponibles dans metadata :", metadata.keys())

# On vérifie si les features sont bien définies
features = metadata.get("features", None)
if features is None:
    print("❌ 'features' est manquant dans metadata.")
else:
    print(f"✅ Nombre de features : {len(features)}")
    print("Extrait des features :", features[:5])  # Affiche les 5 premières



🔍 Clés disponibles dans artifacts : dict_keys(['scaler', 'model', 'metadata'])
🔍 Clés disponibles dans metadata : dict_keys(['optimal_threshold', 'training_date', 'features', 'dtype_example', 'training_stats', 'performance_metrics', 'training_strategy'])
✅ Nombre de features : 85
Extrait des features : ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_CREDIT', 'ANNUITY_INCOME_PERC']


In [6]:
import pickle

# Charge le fichier artifact contenant les informations
artifact_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Charger les artifacts
with open(artifact_path, 'rb') as f:
    artifacts = pickle.load(f)

# Afficher le type de l'objet chargé pour mieux comprendre sa structure
print("🔍 Type des artifacts :", type(artifacts))

# Afficher un extrait des données pour comprendre la structure
print("🔍 Extrait des artifacts :", artifacts[:5])  # Affiche les 5 premiers éléments si c'est un tableau
print("longueur du tableau" ,len(artifacts))


🔍 Type des artifacts : <class 'numpy.ndarray'>
🔍 Extrait des artifacts : ['EXT_SOURCE_1' 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'AMT_CREDIT'
 'ANNUITY_INCOME_PERC']
longueur du tableau 85


In [9]:
import pandas as pd

# Suppose que client_data est un DataFrame que tu veux prédire
TEST_SAMPLE_FOR_API = "../backend/data/test_1000_sample_for_api.csv"
TEST_SAMPLE_WITH_TARGET = "../backend/data/test_1000_sample_with_target.csv"

df_test = pd.read_csv(TEST_SAMPLE_WITH_TARGET)
df_api = pd.read_csv(TEST_SAMPLE_FOR_API)

client_id = df_test["SK_ID_CURR"]
client_data = df_api[df_api["SK_ID_CURR"] == client_id].drop(columns=["SK_ID_CURR"])

# Comparer les features attendues avec les données d'entrée
expected_features = artifacts

# Trouver les colonnes manquantes dans les données d'entrée
missing_cols = set(expected_features) - set(client_data.columns)

# Afficher les résultats
if missing_cols:
    print("⚠️ Colonnes manquantes dans les données d'entrée :", missing_cols)
else:
    print("✅ Toutes les colonnes nécessaires sont présentes dans les données.")


✅ Toutes les colonnes nécessaires sont présentes dans les données.


In [19]:
import joblib

# Charger les artefacts sauvegardés
model_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"
artifacts = joblib.load(model_path)

# Extraire le modèle et le scaler
model = artifacts['model']
scaler = artifacts['scaler']

# Si nécessaire, obtenir les métadonnées
metadata = artifacts['metadata']
optimal_threshold = metadata['optimal_threshold']
features = metadata['features']

# Vérifier les données d'entrée (client_data)
client_data = client_data[features]  

# Appliquer le scaler sur les données du client
client_data_scaled = scaler.transform(client_data)

# Appliquer le scaler en conservant les noms de colonnes
client_data_scaled = pd.DataFrame(
    scaler.transform(client_data),
    columns=client_data.columns,
    index=client_data.index
)

# Faire une prédiction sur les données du client
pred_proba = model.predict_proba(client_data_scaled)[:, 1]  # Probabilité de défaut

# Appliquer le seuil optimal pour la prédiction finale
pred = (pred_proba >= optimal_threshold).astype(int)

# Affichage des résultats
print(f"Probabilité de défaut : {pred_proba[0]:.2%}")
print(f"Décision : {'🛑 Refusé' if pred[0] == 1 else '✅ Accepté'}")


Probabilité de défaut : 3.91%
Décision : ✅ Accepté


In [16]:
import pickle
import lightgbm as lgb

# Charge les artifacts depuis le fichier
artifact_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Charger les artifacts
with open(artifact_path, 'rb') as f:
    artifacts = pickle.load(f)

# Vérifier la structure des artifacts
print("🔍 Type des artifacts :", type(artifacts))
print("🔍 Forme des artifacts :", artifacts.shape)  # Vérifie si c'est un tableau NumPy et sa taille

# Afficher un extrait pour mieux comprendre la structure
print("🔍 Extrait des artifacts :", artifacts[:5])  # Affiche les 5 premiers éléments si c'est un tableau


🔍 Type des artifacts : <class 'numpy.ndarray'>
🔍 Forme des artifacts : (85,)
🔍 Extrait des artifacts : ['EXT_SOURCE_1' 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'AMT_CREDIT'
 'ANNUITY_INCOME_PERC']
